I will start with small 2 qubit problems, one solvable and one not.

# The set

In [1]:
solvable_problem = [1,1] #An solvable instance of the partitioning problem to be solved using 2 quibts
unsolvable_problem = [1,3] #An unsolvable instance of the partitioning problem to be solved using 2 quibts

# Qiskit

In [2]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 5.5 MB/s 
     |████████████████████████████████| 18.0 MB 313 kB/s 
     |████████████████████████████████| 240 kB 60.3 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 112 kB 47.3 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 1.6 MB 44.7 MB/s 
     |████████████████████████████████| 943 kB 44.3 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 4.1 MB 34.1 MB/s 
     |████████████████████████████████| 112 kB 44.9 MB/s 
  Created wheel for qiskit: filename=qiskit-0.37.1-py3-none-any.whl size=12067 sha256=9f571ecea6aabbeb7f6ff08e0d937248f60ef69b6f0273ea3a5f22e03d74274c
  Stored in directory: /root/.cache/pip/wheels/1c/3c/a3/bf8c2

In [3]:
from qiskit.algorithms import VQE
from qiskit import Aer, IBMQ
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.providers.aer.noise import NoiseModel
from qiskit.algorithms.optimizers import L_BFGS_B, COBYLA

seed = 9999


## preparing the simulator

In [4]:
def prepare_simulator(simulator_name, optimizer_name, seed):
    algorithm_globals.random_seed = seed
    backend = Aer.get_backend(simulator_name)
    quantum_instance = QuantumInstance(backend=backend, 
                                      shots=8192, 
                                      seed_transpiler=seed, 
                                      seed_simulator=seed
                                      )
    if optimizer_name == "cobyla" or optimizer_name == "COBYLA":
      optimizer = COBYLA(maxiter=60)
    else:
      optimizer = L_BFGS_B(maxiter=60)
    return optimizer, quantum_instance

## preparing the hamiltonian for 2 quibts (Ising model)

In [5]:
import numpy as np
from qiskit.opflow import Z, I

def create_hamiltonian(S):
  n_1 = S[0]
  n_2 = S[1]

  H = n_1*n_1*(I^I) + n_1*n_2*(Z^Z) + n_2*n_1*(Z^Z) + n_2*n_2*(I^I)
  print("========== Ising Model Hamiltonian for Two Qubits ==========\n")
  print(H.to_matrix())
  print()
  return H

## compute minimal eigenvalue

In [6]:
from qiskit.circuit.library import EfficientSU2

def minimum_eigenvalue_by_vqe(num_qubits, ansatz_entanglement, optimizer, quantum_instance, hamiltonian):
    ansatz = EfficientSU2(num_qubits=num_qubits, entanglement=entanglement)
    #ansatz.draw(output='latex', scale=0.5)
    vqe = VQE(ansatz, optimizer, quantum_instance=quantum_instance)
    result = vqe.compute_minimum_eigenvalue(hamiltonian)
    print("minimal eigenvalue: ", format(result.eigenvalue,".3f"))

## compute minimal eigenvalue for solvable problem

In [7]:
from qiskit.circuit.library import EfficientSU2  
num_qubits = 2
entanglement = "linear"
H = create_hamiltonian(solvable_problem)

========== Ising Model Hamiltonian for Two Qubits ==========

[[4.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 4.+0.j]]



### qasm simulator and cobyla optimizer

In [8]:
bfgs_optimizer, quantum_instance = prepare_simulator("qasm_simulator", "cobyla", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)

minimal eigenvalue:  0.000+0.000j


### qasm simulator and bfgs optimizer

returns wrong result :/ expected result: 0.000

In [9]:
from qiskit.tools.visualization import circuit_drawer
bfgs_optimizer, quantum_instance = prepare_simulator("qasm_simulator", "bfgs", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)

minimal eigenvalue:  1.247+0.000j


### aer_statevector simulator and cobyla optimizer

returns wrong result :/ expected result: 0.000

In [10]:
cobyla_optimizer, quantum_instance = prepare_simulator("aer_simulator_statevector", "cobyla", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, cobyla_optimizer, quantum_instance, H)

minimal eigenvalue:  0.002+0.000j


### aer_statevector simulator and bfgs optimizer

In [11]:
bfgs_optimizer, quantum_instance = prepare_simulator("aer_simulator_statevector", "bfgs", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)

minimal eigenvalue:  0.000+0.000j


## compute minimal eigenvalue for unsolvable problem

In [12]:
from qiskit.circuit.library import EfficientSU2  
num_qubits = 2
entanglement = "linear"
H = create_hamiltonian(unsolvable_problem)

========== Ising Model Hamiltonian for Two Qubits ==========

[[16.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  4.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  4.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j 16.+0.j]]



### qasm simulator and cobyla optimizer

In [13]:
bfgs_optimizer, quantum_instance = prepare_simulator("qasm_simulator", "cobyla", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)

minimal eigenvalue:  4.000+0.000j


### qasm simulator and bfgs optimizer

returns wrong result :/ expected result: 4.000

In [14]:
from qiskit.tools.visualization import circuit_drawer
bfgs_optimizer, quantum_instance = prepare_simulator("qasm_simulator", "bfgs", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)

minimal eigenvalue:  7.740+0.000j


### aer_statevector simulator and cobyla optimizer

In [15]:
cobyla_optimizer, quantum_instance = prepare_simulator("aer_simulator_statevector", "cobyla", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, cobyla_optimizer, quantum_instance, H)

minimal eigenvalue:  4.004+0.000j


### aer_statevector simulator and bfgs optimizer

In [16]:
bfgs_optimizer, quantum_instance = prepare_simulator("aer_simulator_statevector", "bfgs", seed) 
minimum_eigenvalue_by_vqe(num_qubits, entanglement, bfgs_optimizer, quantum_instance, H)

minimal eigenvalue:  4.000+0.000j


## General Form

In [17]:
solvable_problem_4_qubits = [1,2,3,2]
unsolvable_problem_4_qubits = [1,2,4,2]

### preparing the hamiltonian for n quibts (Ising model)

In [18]:
import numpy as np
from qiskit.opflow import Z, I

def create_general_hamiltonian(S):
  num_qubits = len(S)
  sum = 0
  sigmas_list = []
  for qubit in range(num_qubits):
    if qubit == 0:
       sigma = Z
    else:
       sigma = I
    for i in range(1,num_qubits):
      if i == qubit:
        sigma = sigma^Z
      else:
        sigma = sigma^I
    sigmas_list.append(sigma)  

  for i in range(len(sigmas_list)):
    for j in range(len(sigmas_list)):
      sum += S[i]*S[j]* sigmas_list[i]@sigmas_list[j]
  print("========== Ising Model Hamiltonian ==========\n")
  print(sum.to_matrix())
  print()
  return sum

### Example: solvable 4 qubit problem

In [19]:
H = create_general_hamiltonian(solvable_problem_4_qubits)
bfgs_optimizer, quantum_instance = prepare_simulator("aer_simulator_statevector", "bfgs", seed) 
minimum_eigenvalue_by_vqe(len(solvable_problem_4_qubits), entanglement, bfgs_optimizer, quantum_instance, H)

========== Ising Model Hamiltonian ==========

[[64.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j 16.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  4.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  4.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j 16.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  4.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+

### Example: unsolvable 4 qubit problem

In [20]:
H = create_general_hamiltonian(unsolvable_problem_4_qubits)
bfgs_optimizer, quantum_instance = prepare_simulator("aer_simulator_statevector", "bfgs", seed) 
minimum_eigenvalue_by_vqe(len(solvable_problem_4_qubits), entanglement, bfgs_optimizer, quantum_instance, H)

========== Ising Model Hamiltonian ==========

[[81.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j 25.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  9.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j 25.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  9.+0.j  0.+0.j  0.+0.j
   0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+